In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms
from sklearn.metrics import classification_report
import os
import copy
import numpy as np

In [8]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'Yasinetski_dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),data_transforms[x])for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=5,shuffle=True, num_workers=2)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)
print(dataset_sizes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

['camponotus_vagus', 'lasius_flavus', 'myrmecia_gulosa', 'odontomachus_bauri']
{'train': 476, 'val': 80}


In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [11]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 4)

model_conv = model_conv.to(device)


criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7)

In [12]:
model_conv = train_model(model_conv, criterion, optimizer_conv,exp_lr_scheduler, num_epochs=25)

Epoch 1/25
----------
train Loss: 1.1184 Acc: 0.4874
val Loss: 0.6556 Acc: 0.7625

Epoch 2/25
----------
train Loss: 0.7574 Acc: 0.7080
val Loss: 0.5017 Acc: 0.8000

Epoch 3/25
----------
train Loss: 0.7311 Acc: 0.6954
val Loss: 0.4997 Acc: 0.8000

Epoch 4/25
----------
train Loss: 0.7801 Acc: 0.6765
val Loss: 0.5426 Acc: 0.8250

Epoch 5/25
----------
train Loss: 0.6505 Acc: 0.7290
val Loss: 0.3481 Acc: 0.8625

Epoch 6/25
----------
train Loss: 0.7116 Acc: 0.7269
val Loss: 0.4496 Acc: 0.8000

Epoch 7/25
----------
train Loss: 0.6650 Acc: 0.7185
val Loss: 0.5025 Acc: 0.8000

Epoch 8/25
----------
train Loss: 0.5559 Acc: 0.7857
val Loss: 0.3489 Acc: 0.8750

Epoch 9/25
----------
train Loss: 0.5290 Acc: 0.7899
val Loss: 0.3224 Acc: 0.8875

Epoch 10/25
----------
train Loss: 0.5460 Acc: 0.7983
val Loss: 0.3942 Acc: 0.8375

Epoch 11/25
----------
train Loss: 0.4714 Acc: 0.8361
val Loss: 0.3483 Acc: 0.8875

Epoch 12/25
----------
train Loss: 0.5634 Acc: 0.7836
val Loss: 0.3190 Acc: 0.8750

E

In [13]:
y_pred = []
y_true = []
with torch.no_grad():
    model_conv.eval()
    for inputs, labels in dataloaders['val']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_conv(inputs)
        _, outputs = torch.max(outputs, dim = 1)
        y_pred.append(outputs.cpu().numpy())
        y_true.append(labels.cpu().numpy())
y_pred = [a.squeeze().tolist() for a in y_pred]
y_true = [a.squeeze().tolist() for a in y_true]
y_pred = np.array(y_pred)
y_true = np.array(y_true)
y_pred = y_pred.flatten()
y_true = y_true.flatten()

In [14]:
print(classification_report(y_true,y_pred,target_names=class_names))

                    precision    recall  f1-score   support

  camponotus_vagus       0.90      0.90      0.90        20
     lasius_flavus       0.91      1.00      0.95        20
   myrmecia_gulosa       1.00      0.80      0.89        20
odontomachus_bauri       0.82      0.90      0.86        20

          accuracy                           0.90        80
         macro avg       0.91      0.90      0.90        80
      weighted avg       0.91      0.90      0.90        80

